# Analyzing Book Databases

This report will analyze 4 databases using SQL in order to generate a value proposition for a new app geared towards book lovers. 
The 4 databases are:
- books:
    book_id
    author_id
    title
    num_pages — number of pages
    publication_date
    publisher_id

- authors:
    Contains data on authors:
    author_id
    author

- publishers:
    Contains data on publishers:
    publisher_id
    publisher

- ratings:
    Contains data on user ratings:
    rating_id
    book_id
    username — the name of the user who rated the book
    rating

- reviews:
    Contains data on customer reviews:
    review_id
    book_id
    username — the name of the user who reviewed the book
    text — the text of the review
    
 

In [1]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine




In [2]:
db_config = {'user': 'praktikum_student',         # user name
             'pwd': 'Sdf4$2;d-d30pp', # password
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432,              # connection port
             'db': 'data-analyst-final-project-db'}          # the name of the data base

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                                     db_config['pwd'],
                                                                       db_config['host'],
                                                                       db_config['port'],
                                                                       db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

Let's first study each of the individual tables by printing the first few rows to become more familiar with what they contain.

In [3]:
#the authors database
query= 'SELECT *, COUNT(*) OVER() AS table_size FROM authors LIMIT 1'

authors= pd.io.sql.read_sql(query, con = engine)
authors.head()

,author_id,author,table_size
0,1,A.S. Byatt,636


In [4]:
#the publishers database
query= 'SELECT *, COUNT(*) OVER() AS table_size FROM publishers LIMIT 1'

publishers= pd.io.sql.read_sql(query, con = engine)
publishers.head()

,publisher_id,publisher,table_size
0,1,Ace,340


In [5]:
#the ratings database
query= 'SELECT *, COUNT(*) OVER() AS table_size FROM ratings LIMIT 1'

ratings= pd.io.sql.read_sql(query, con = engine)
ratings.head()

,rating_id,book_id,username,rating,table_size
0,1,1,ryanfranco,4,6456


In [6]:
#the reviews database
query= 'SELECT *, COUNT(*) OVER() AS table_size FROM reviews LIMIT 1'

reviews= pd.io.sql.read_sql(query, con = engine)
reviews.head()

,review_id,book_id,username,text,table_size
0,1,1,brandtandrea,Mention society tell send professor analysis. ...,2793


In [7]:
#the books database
query= 'SELECT *, COUNT(*) OVER() AS table_size FROM books LIMIT 1'

books= pd.io.sql.read_sql(query, con = engine)
books

,book_id,author_id,title,num_pages,publication_date,publisher_id,table_size
0,1,546,'Salem's Lot,594,2005-11-01,93,1000


There are a few relationships between the tables, namely the book_id field that is present in the books, ratings and reviews tables. As well as the publisher_id that is found in the book and publisher tables. Author_id is found in the book and author tables. 

The ratings table is our largest with a size of 6456.

# Task 1
My first task is to find the number of books released after January 1, 2000.


In [10]:
#TASK 1
query="SELECT COUNT(book_id) AS count FROM books WHERE publication_date >'2000-01-01'"

book_count=pd.io.sql.read_sql(query, con = engine)
book_count

,count
0,819


## Task 1 Conclusion
There are 819 books that were published after January 1, 2008. That is the majority of the books database that holds 1000 books.

# Task 2
The next query will find the number of user reviews and the average rating for each book. 

In [11]:
#TASK 2
query=''' 
SELECT books.book_id AS book_id,
    COUNT (DISTINCT reviews.review_id) AS count_reviews,
    AVG(ratings.rating) AS avg_rating
    FROM books INNER JOIN reviews ON reviews.book_id = books.book_id 
    INNER JOIN ratings ON ratings.book_id= books.book_id 
    GROUP BY books.book_id
    ORDER BY books.book_id
'''
rev_review= pd.io.sql.read_sql(query, con=engine)
rev_review

,book_id,count_reviews,avg_rating
0,1,2,3.666667
1,2,1,2.500000
2,3,3,4.666667
3,4,2,4.500000
4,5,4,4.000000
...,...,...,...
989,996,3,3.666667
990,997,3,3.400000
991,998,4,3.200000
992,999,2,4.500000


## Task 2 Conclusion
We have found the number of reviews and average rating for each book. For example book_id 1 has 2 reviews with an average rating of 3.6667.

# Task 3
Next I will identify the publisher that has released the greatest number of books with more than 50 pages

In [13]:
#TASK 3
query= '''
SELECT books.publisher_id,
publishers.publisher,
COUNT(book_id) AS count 
FROM books INNER JOIN publishers ON publishers.publisher_id = books.publisher_id
WHERE num_pages > 50 
GROUP BY books.publisher_id, publishers.publisher
ORDER BY COUNT(book_id) DESC 
LIMIT 3
'''

greatest_pub= pd.io.sql.read_sql(query, con=engine)
greatest_pub

,publisher_id,publisher,count
0,212,Penguin Books,42
1,309,Vintage,31
2,116,Grand Central Publishing,25


## Task 3 Conclusion
The top three publishers that have released the greatest number of books with more than 50 pages is above, with Penguin Books (publisher_id 212) having released the greatest number with 42. 

# Task 4
Identify the author with the highest average book rating: look only at books with at least 50 ratings. 

In [14]:
#TASK 4
query='''
SELECT Sub.author,
AVG(Sub.avg_rating) AS avg_rating,
COUNT(DISTINCT Sub.book_id)
FROM
    (SELECT books.book_id,
    authors.author AS author,
    books.author_id,
    AVG(ratings.rating) AS avg_rating
    FROM books INNER JOIN ratings ON ratings.book_id= books.book_id 
    INNER JOIN authors ON authors.author_id = books.author_id
    GROUP BY authors.author, books.book_id
    HAVING COUNT(DISTINCT ratings.rating_id) >=50 
    ORDER BY avg_rating DESC) AS Sub
GROUP BY Sub.author
ORDER BY avg_rating DESC
LIMIT 3
'''
rev_4=pd.io.sql.read_sql(query, con=engine)
rev_4

,author,avg_rating,count
0,J.K. Rowling/Mary GrandPré,4.283844,4
1,Markus Zusak/Cao Xuân Việt Khương,4.264151,1
2,J.R.R. Tolkien,4.258446,2


## Task 4 Conclusion

The top 3 authors (with at least 50 ratings) with the highest average book ratings are shown above with J.K. Rowling/ Mary GrandePre  having the highest rating of 4.28

# Task 5
Find the average number of text reviews among users who rated more than 50 books

In [15]:
query= '''
SELECT AVG(Sub.text_count)
FROM
    (SELECT reviews.username,
    COUNT(DISTINCT ratings.rating_id) AS rating_count,
    COUNT(DISTINCT reviews.text) AS text_count
    FROM reviews INNER JOIN ratings ON ratings.username=reviews.username
    GROUP BY reviews.username
    HAVING COUNT(DISTINCT ratings.rating_id) > 50) AS Sub
'''
rev5=pd.io.sql.read_sql(query, con=engine)
rev5

,avg
0,24.333333


## Task 5 Conclusion

I've used a subquery to fulfill this task and have found that the average number of text reviews among those users is about 24.3

# Conclusion
The goal of this report was to analyze 4 databases that were geared towards book publication. We were able to find the number of books published after January 1st, 2000 (819), the number of user reviews and the average rating for each book, identify the publisher that has released the greatest number of books with more than 50 pages (Penguin Books), find the author with the highest average book rating, looking only at books with at least 50 ratings (J.K. Rowling) and find the average number of text reviews among users who rated more than 50 books (24.33). 

With these findings we will be able to create a value proposition for a new app for avid book readers. 